# Bundesliga Tables in a world where the three-point-rule was never introduced

In this notebook we recalculate the final standings of the Bundesliga seasons 95/96 - 22/23. Instead of the three points per win, that were introduced before the 95/96 season, we calculate the tables using the previously used 2 points per win. 

Of course, this completely neglects all kinds of factors like different tactics and psychological effects based on different standings during the season. It's just a bit of "what if?" fun. 🙂⚽️

The tables below show the alternate final standings with the alternate 2-rule point sums in the "points-2" columnn and the difference in final position of each team shown in the "diff" column.

There are two "robbed" Meisterschalen and some relegations that could have been prevented. There is also a season in which it would not have made a difference. In general, you can see that especially lower-table teams that earned lots of (probably hard fought) draws are at a disadvantage with the 3-point-rule. 

In [1]:
import requests;
import pandas as pd;
import numpy as np;
#import seaborn as sns;
pd.options.mode.copy_on_write = True

In [2]:
# Set to either teamName or shortName
displayName = "teamName"
green = "#a1e3b6"
red = "#f2b4aa"

styleFunc = lambda x: ["background: {}".format(green) if (isinstance(v, int) and v > 0) else ("background: {}".format(red) if (isinstance(v, int) and v < 0) else "") for v in x]

## Seasons: 1995/96 until 2008/09

For these seasons, we have to calculate the tables from the match history dataset first as I did not find any suitable API. 

The source of the dataset can be found at: https://github.com/thomascamminady/AllBundesligaGamesEver.
Note that the dataset has some duplicate entries we need to drop. Also the author of the dataset has swapped the points awarded columns for home and guest teams which led to some major confusion as to why the tables did not add up to the accurate point sums 😵‍💫

In [3]:
path = "./dataset.csv"
gamesTable = pd.read_csv(path, sep=";")
selectionTable = gamesTable[(gamesTable["SeasonFrom"].astype(int)>=1995) & (gamesTable["League"]=="1. Bundesliga")][["SeasonFrom","Home", "Guest", "Score90Home", "Score90Guest", "PointsHome", "PointsGuest"]]

In [4]:
for year in range(1995,2008):
    print("\n--- Season {}/{} ---\n".format(year, year+1))
    
    # Get all matches of the season
    season = selectionTable[selectionTable["SeasonFrom"].astype(int)==year]
    season.drop_duplicates(inplace=True)
    season["PointsHome-2"] = np.where(season["PointsHome"]==3, 2, season["PointsHome"])
    season["PointsGuest-2"] = np.where(season["PointsGuest"]==3, 2, season["PointsGuest"])

    # Select teams that played in that season
    allTeams = season["Home"].unique()
    allTeams = pd.DataFrame(data=allTeams, columns=["teamName"])

    pointsPerTeam = []
    points2PerTeam = []
    goalDiffPerTeam = []
    
    # calculate every teams goaldiff, points (3perWin) and alternate points (2perWin)
    for i, table in allTeams.iterrows():
        homeGoalDiff = season[season["Home"] == table[0]]["Score90Home"].sum() - season[season["Home"] == table[0]]["Score90Guest"].sum()
        guestGoalDiff = season[season["Guest"] == table[0]]["Score90Guest"].sum() - season[season["Guest"] == table[0]]["Score90Home"].sum()
        goalDiffPerTeam.append(homeGoalDiff+guestGoalDiff)
        # Here we have to swap the columns because of an error in the dataset 
        homePoints = season[season["Home"] == table[0]]["PointsGuest"].sum()
        guestPoints = season[season["Guest"] == table[0]]["PointsHome"].sum()
        totalPoints = (homePoints+guestPoints)
        pointsPerTeam.append(totalPoints)
        # calculate alternate points per team (2 pts per win). again swap the columns 
        homePoints2 = season[season["Home"] == table[0]]["PointsGuest-2"].sum()
        guestPoints2 = season[season["Guest"] == table[0]]["PointsHome-2"].sum()
        totalPoints2 = (homePoints2+guestPoints2)
        points2PerTeam.append(totalPoints2)

    # calculate the final tables and differences
    allTeams["points"] = pointsPerTeam
    allTeams["goalDiff"] = goalDiffPerTeam
    allTeams["points-2"] = points2PerTeam
    allTeams.sort_values(by=["points", "goalDiff"], ascending=False, inplace = True)
    allTeams.reset_index(inplace = True, drop=True)
    allTeams.reset_index(inplace = True, drop=False)
    allTeams.sort_values(by=["points-2", "goalDiff"], ascending=False, inplace = True)
    allTeams.reset_index(inplace = True, drop=True)
    
    allTeams["diff"] = allTeams["index"]- allTeams.index

    display(allTeams.style.apply(styleFunc, subset=pd.IndexSlice[:, ['diff']]))


--- Season 1995/1996 ---



,index,teamName,points,goalDiff,points-2,diff
0,0,Borussia Dortmund,68,38,49,0
1,1,Bayern München,62,20,43,0
2,2,FC Schalke 04,56,9,42,0
3,4,Hamburger SV,50,5,38,1
4,3,Bor. Mönchengladbach,53,1,38,-1
5,6,Karlsruher SC,48,6,36,1
6,5,Hansa Rostock,49,4,36,-1
7,7,1860 München,45,6,34,0
8,8,Werder Bremen,44,-3,34,0
9,9,VfB Stuttgart,43,-3,33,0



--- Season 1996/1997 ---



,index,teamName,points,goalDiff,points-2,diff
0,0,Bayern München,71,34,51,0
1,1,Bayer 04 Leverkusen,69,28,48,0
2,2,Borussia Dortmund,63,22,44,0
3,3,VfB Stuttgart,61,38,43,0
4,4,VfL Bochum,53,3,39,0
5,5,Karlsruher SC,49,11,36,0
6,6,1860 München,49,0,36,0
7,7,Werder Bremen,48,1,34,0
8,8,MSV Duisburg,45,-5,33,0
9,11,FC Schalke 04,43,-5,32,2



--- Season 1997/1998 ---



...

,index,teamName,points,goalDiff,points-2,diff
0,0,Bayern München,75,35,53,0
1,1,Werder Bremen,70,42,49,0
2,2,Hamburger SV,68,23,47,0
3,3,FC Schalke 04,61,16,45,0
4,4,Bayer 04 Leverkusen,52,15,38,0
5,5,Hertha BSC,48,4,36,0
6,6,Borussia Dortmund,46,3,35,0
7,8,VfB Stuttgart,43,-2,34,1
8,7,1. FC Nürnberg,44,-2,32,-1
9,9,Bor. Mönchengladbach,42,-8,32,0



--- Season 2006/2007 ---



,index,teamName,points,goalDiff,points-2,diff
0,0,VfB Stuttgart,70,24,49,0
1,1,FC Schalke 04,68,21,47,0
2,2,Werder Bremen,66,36,46,0
3,3,Bayern München,60,15,42,0
4,5,1. FC Nürnberg,48,11,37,1
5,4,Bayer 04 Leverkusen,51,5,36,-1
6,6,Hamburger SV,45,6,35,0
7,7,VfL Bochum,45,-1,32,0
8,8,Borussia Dortmund,44,-2,32,0
9,9,Hertha BSC,44,-5,32,0



--- Season 2007/2008 ---



,index,teamName,points,goalDiff,points-2,diff
0,0,Bayern München,76,47,54,0
1,1,Werder Bremen,66,30,46,0
2,2,FC Schalke 04,64,23,46,0
3,3,Hamburger SV,54,21,40,0
4,4,VfL Wolfsburg,54,12,39,0
5,6,Bayer 04 Leverkusen,51,17,36,1
6,5,VfB Stuttgart,52,0,36,-1
7,7,Hannover 96,49,-2,36,0
8,8,Eintracht Frankfurt,46,-7,34,0
9,9,Hertha BSC,44,-5,32,0


## Seasons 2008/09 until 2022/23

For these seasons I found a nice API allowing me to fetch the final tables. 
https://openligadb.de

In [5]:
for year in range(2008,2023):
    print("\n--- Season {}/{} ---\n".format(year, year+1))
    # fetch table and format 
    apiUrl = "https://api.openligadb.de/getbltable/bl1/{y}".format(y = year)
    response = requests.get(apiUrl)
    responseData = response.json();
    table = pd.DataFrame(responseData)
    tableShort = table[[displayName, "points", "won", "draw", "lost", "goalDiff"]]
    # calc alternate Points and create second table
    tableShort["points-2"] = tableShort["won"]*2 + tableShort["draw"]
    alternateTable = tableShort.copy()
    alternateTable.sort_values(by=['points-2', "goalDiff"], ascending=False, inplace = True)
    alternateTable.reset_index(inplace=True)
    alternateTable["diff"] = alternateTable["index"]- alternateTable.index
    
    display(alternateTable.style.apply(styleFunc, subset=pd.IndexSlice[:, ['diff']]))
    
    


--- Season 2008/2009 ---



,index,teamName,points,won,draw,lost,goalDiff,points-2,diff
0,0,VfL Wolfsburg,69,21,6,7,39,48,0
1,1,FC Bayern München,67,20,7,7,29,47,0
2,2,VfB Stuttgart,64,19,7,8,20,45,0
3,5,Borussia Dortmund,59,15,14,5,23,44,2
4,3,Hertha BSC,63,19,6,9,7,44,-1
5,4,Hamburger SV,61,19,4,11,2,42,-1
6,6,TSG 1899 Hoffenheim,55,15,10,9,14,40,0
7,7,FC Schalke 04,50,14,8,12,12,36,0
8,8,Bayer Leverkusen,49,14,7,13,13,35,0
9,9,Werder Bremen,45,12,9,13,14,33,0



--- Season 2009/2010 ---



,index,teamName,points,won,draw,lost,goalDiff,points-2,diff
0,0,FC Bayern München,70,20,10,4,41,50,0
1,1,FC Schalke 04,65,19,8,7,22,46,0
2,2,Werder Bremen,61,17,10,7,31,44,0
3,3,Bayer Leverkusen,59,15,14,5,27,44,0
4,4,Borussia Dortmund,57,16,9,9,12,41,0
5,5,VfB Stuttgart,55,15,10,9,10,40,0
6,6,Hamburger SV,52,13,13,8,15,39,0
7,7,VfL Wolfsburg,50,14,8,12,6,36,0
8,8,1. FSV Mainz 05,47,12,11,11,-6,35,0
9,9,Eintracht Frankfurt,46,12,10,12,-7,34,0



--- Season 2010/2011 ---



...

,index,teamName,points,won,draw,lost,goalDiff,points-2,diff
0,0,FC Bayern München,78,24,6,4,55,54,0
1,1,RB Leipzig,65,19,8,7,28,46,0
2,2,Borussia Dortmund,64,20,4,10,29,44,0
3,3,VfL Wolfsburg,61,17,10,7,24,44,0
4,4,Eintracht Frankfurt,60,16,12,6,16,44,0
5,5,Bayer Leverkusen,52,14,10,10,14,38,0
6,6,1. FC Union Berlin,50,12,14,8,7,38,0
7,7,Borussia Mönchengladbach,49,13,10,11,8,36,0
8,8,VfB Stuttgart,45,12,9,13,1,33,0
9,9,SC Freiburg,45,12,9,13,0,33,0



--- Season 2021/2022 ---



,index,teamName,points,won,draw,lost,goalDiff,points-2,diff
0,0,FC Bayern München,77,24,5,5,60,53,0
1,1,Borussia Dortmund,69,22,3,9,33,47,0
2,2,Bayer Leverkusen,64,19,7,8,33,45,0
3,3,RB Leipzig,58,17,7,10,35,41,0
4,4,1. FC Union Berlin,57,16,9,9,6,41,0
5,5,SC Freiburg,55,15,10,9,12,40,0
6,6,1. FC Köln,52,14,10,10,3,38,0
7,7,1. FSV Mainz 05,46,13,7,14,5,33,0
8,8,TSG 1899 Hoffenheim,46,13,7,14,-2,33,0
9,9,Borussia Mönchengladbach,45,12,9,13,-7,33,0



--- Season 2022/2023 ---



,index,teamName,points,won,draw,lost,goalDiff,points-2,diff
0,0,FC Bayern München,71,21,8,5,54,50,0
1,1,Borussia Dortmund,71,22,5,7,39,49,0
2,2,RB Leipzig,66,20,6,8,23,46,0
3,3,1. FC Union Berlin,62,18,8,8,13,44,0
4,4,SC Freiburg,59,17,8,9,7,42,0
5,6,Eintracht Frankfurt,50,13,11,10,6,37,1
6,7,VfL Wolfsburg,49,13,10,11,9,36,1
7,5,Bayer Leverkusen,50,14,8,12,8,36,-2
8,8,1. FSV Mainz 05,46,12,10,12,-1,34,0
9,9,Borussia Mönchengladbach,43,11,10,13,-3,32,0
